In [1]:

%matplotlib inline
from __future__ import unicode_literals, print_function, division

import unicodedata
import string
import re
import time
import os
import glob
import csv
import copy

from pathlib import Path
import pandas as pd
import math
import random
import pickle
import gc
import pretty_midi
from midiutil.MidiFile import MIDIFile
from music21 import *
from io import open
import numpy as np
import matplotlib
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from itertools import groupby
import librosa

from midi2audio import FluidSynth

import scipy

import datetime
from pathlib import Path
import math
matplotlib.use('Agg')
# matplotlib.use("QtAgg")
import ffmpeg
#conda install -c conda-forge ffmpeg-python

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, writers
plt.rcParams['animation.ffmpeg_path'] = '/home/ilc/anaconda3/bin/ffmpeg'#'/usr/local/bin/ffmpeg'

import numpy as np
import subprocess as sp
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip

import pretty_midi
import numpy as np
import pickle

import torch
import torch.nn as nn
from torch import Tensor
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
# from torch.nn import TransformerEncoder, TransformerEncoderLayer
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
change_fps = 40 #40

In [3]:
this_performer = "vio01"
performaer_list = ["vio01"] #, "vio02", "vio03", "vio04", "vio05"
test_pieces = ['Elgar_S1_T1', 'Flower_S1_T1']
# test_pieces = ['Elgar_S1_T1', 'Elgar_S1_T2',
#                    'Flower_S1_T1', 'Flower_S1_T2',
#                    'Mend_S1_T1', 'Mend_S1_T2',
#                    'Mozart1_S1_T1', 'Mozart1_S1_T2',
#                    'Mozart2_S1_T1', 'Mozart2_S1_T2']
# piece_list = ["Bach1_S1_T1", "Bach1_S1_T2", "Bach2_S1_T1", "Bach2_S1_T2", "Bach2_S2_T1", "Bach2_S2_T2", "Beeth1_S1_T1", "Beeth1_S1_T2", "Beeth2_S1_T1", "Beeth2_S1_T2", "Elgar_S1_T1", "Elgar_S1_T2", "Flower_S1_T1", "Flower_S1_T2", "Mend_S1_T1", "Mend_S1_T2", "Mozart1_S1_T1", "Mozart1_S1_T2", "Mozart2_S1_T1", "Mozart2_S1_T2", "Wind_S1_T1", "Wind_S1_T2"]

In [4]:
# UoE_mocap_path = "./UoE_violin_midi/performance_motion/"

In [5]:
def read_midi(filename):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(filename)

    # Get the piano roll representation of the MIDI file
    # fs = 40
    piano_roll = midi_data.get_piano_roll(fs=change_fps) #40fps #250fps
    piano_roll[piano_roll > 0] = 1

    return piano_roll

def audio_preprocess(audio_path, specific_fps):
    n_fft = 4096
    hop = int(44000/specific_fps)  # 1102.5 -> 40fps #882 -> 50fps
    y, sr = librosa.load(audio_path, sr=44000)  # 44000 for divide 40
    print("y.shape", y.shape)
    print("sample rate: ", sr)
    mfcc = librosa.feature.mfcc(
        y=y, sr=sr, n_fft=n_fft, hop_length=hop, n_mfcc=13)
    y = np.where(y == 0, 1e-10, y)
    energy = np.log(librosa.feature.rms(
        y=y, frame_length=n_fft, hop_length=hop, center=True))
    mfcc_energy = np.vstack((mfcc, energy))
    mfcc_delta = librosa.feature.delta(mfcc_energy)

    sgram = librosa.stft(y, n_fft=n_fft, hop_length=hop)
    sgram_mag, _ = librosa.magphase(sgram)
    mel_scale_sgram = librosa.feature.melspectrogram(
        S=sgram_mag, sr=sr)

    print("mfcc_energy", mfcc_energy.shape)
    print("mfcc_delta", mfcc_delta.shape)
    print("mel_scale_sgram", mel_scale_sgram.shape)

    aud = np.vstack((mfcc_energy, mfcc_delta, mel_scale_sgram)).T

    print("hop:", hop)
    print("aud:", aud.shape)
    return aud

def read_motion(motion_path):
    file = open(motion_path, 'rb')
    motion_data = pickle.load(file)
    file.close()

    return motion_data
    

In [6]:
UoE_violin_midi_path = "./UoE_violin_midi/violin_midi/vio01/"
UoE_violin_audio_path = "./UoE_violin_midi/performance_audio/vio01/"
UoE_violin_motion_path = "./preprocessed_data_save_cross/motion/"
#midi_path_list = glob.glob(UoE_violin_midi_path + "/" + this_performer + "/*.mid")

piano_roll_dictionary = {}
piano_roll_dictionary[this_performer] = {}
piano_roll_length_list = []

audio_dictionary = {}
audio_dictionary[this_performer] = {}
audio_len_list = []

motion_dictionary = {}
motion_dictionary[this_performer] = {}
motion_len_list = []

for name_code in test_pieces:#midi_path_list:
    # read MIDI
    read_piano_roll = read_midi(UoE_violin_midi_path + "vio01_" + name_code + ".mid")
    midi_len = len(read_piano_roll[0])
    print("seq_len of midi data:", len(read_piano_roll[0]))
    piano_roll_length_list.append(len(read_piano_roll[0]))
    print("midi data shape: ", read_piano_roll.T.shape)

    # read Audio
    read_audio_mfcc = audio_preprocess(UoE_violin_audio_path + "vio01_" + name_code + ".wav", change_fps)
    audio_len = len(read_audio_mfcc)
    print("seq_len of audio data:", len(read_audio_mfcc))
    audio_len_list.append(len(read_audio_mfcc))
    print("audio data shape: ", read_audio_mfcc.shape)
    
    # read Motion
    motion_input = read_motion(UoE_violin_motion_path + "vio01_" + name_code + "_motion_data.pkl")
    motion_len = len(motion_input)
    print("seq_len of motion data:", len(motion_input))
    motion_len_list.append(len(motion_input))
    print("motion data shape: ", motion_input.shape)
    
    cut_len = min(midi_len, audio_len, motion_len)

    piano_roll_dictionary[this_performer][name_code] = read_piano_roll.T[:cut_len]
    audio_dictionary[this_performer][name_code] = read_audio_mfcc[:cut_len]
    motion_dictionary[this_performer][name_code] = motion_input[:cut_len]
    
    print("[cut]midi data shape: ", piano_roll_dictionary[this_performer][name_code].shape)
    print("[cut]audio data shape: ", audio_dictionary[this_performer][name_code].shape)
    print("[cut]motion data shape: ", motion_dictionary[this_performer][name_code].shape)
    
    print("========")
    # read_mfcc_transpose = read_audio_mfcc
    # audio_len = len(read_mfcc_transpose)

# print(piano_roll_dictionary)

seq_len of midi data: 7355
midi data shape:  (7355, 128)


y.shape (8291800,)
sample rate:  44000
mfcc_energy (14, 7539)
mfcc_delta (14, 7539)
mel_scale_sgram (128, 7539)
hop: 1100
aud: (7539, 156)
seq_len of audio data: 7539
audio data shape:  (7539, 156)
seq_len of motion data: 7524
motion data shape:  (7524, 115)
[cut]midi data shape:  (7355, 128)
[cut]audio data shape:  (7355, 156)
[cut]motion data shape:  (7355, 115)
seq_len of midi data: 4618
midi data shape:  (4618, 128)
y.shape (5247880,)
sample rate:  44000
mfcc_energy (14, 4771)
mfcc_delta (14, 4771)
mel_scale_sgram (128, 4771)
hop: 1100
aud: (4771, 156)
seq_len of audio data: 4771
audio data shape:  (4771, 156)
seq_len of motion data: 4695
motion data shape:  (4695, 115)
[cut]midi data shape:  (4618, 128)
[cut]audio data shape:  (4618, 156)
[cut]motion data shape:  (4618, 115)


In [7]:
body_joint_dict = {"RFHD_X":0,"RFHD_Y":1,"RFHD_Z":2, # head
              "LFHD_X":3,"LFHD_Y":4,"LFHD_Z":5,
              "RBHD_X":6,"RBHD_Y":7,"RBHD_Z":8,
              "LBHD_X":9,"LBHD_Y":10,"LBHD_Z":11,
              "C7_X":12,"C7_Y":13,"C7_Z":14,
              "T10_X":15,"T10_Y":16,"T10_Z":17,
              "CLAV_X":18,"CLAV_Y":19,"CLAV_Z":20,
              "STRN_X":21,"STRN_Y":22,"STRN_Z":23,
              "RSHO_X":24,"RSHO_Y":25,"RSHO_Z":26,
              "RELB_X":27,"RELB_Y":28,"RELB_Z":29,
              "RWRA_X":30,"RWRA_Y":31,"RWRA_Z":32,
              "RWRB_X":33,"RWRB_Y":34,"RWRB_Z":35,
              "RFIN_X":36,"RFIN_Y":37,"RFIN_Z":38,
              "LSHO_X":39,"LSHO_Y":40,"LSHO_Z":41,
              "LELB_X":42,"LELB_Y":43,"LELB_Z":44,
              "LWRA_X":45,"LWRA_Y":46,"LWRA_Z":47,
              "LWRB_X":48,"LWRB_Y":49,"LWRB_Z":50,
              "LFIN_X":51,"LFIN_Y":52,"LFIN_Z":53,
              "RASI_X":54,"RASI_Y":55,"RASI_Z":56,#(18*3) left waist
              "LASI_X":57,"LASI_Y":58,"LASI_Z":59,#(19*3) right waist
              "RPSI_X":60,"RPSI_Y":61,"RPSI_Z":62,
              "LPSI_X":63,"LPSI_Y":64,"LPSI_Z":65,
              "RKNE_X":66,"RKNE_Y":67,"RKNE_Z":68,
              "RHEE_X":69,"RHEE_Y":70,"RHEE_Z":71,
              "RTOE_X":72,"RTOE_Y":73,"RTOE_Z":74,
              "RANK_X":75,"RANK_Y":76,"RANK_Z":77,
              "LKNE_X":78,"LKNE_Y":79,"LKNE_Z":80,
              "LHEE_X":81,"LHEE_Y":82,"LHEE_Z":83,
              "LTOE_X":84,"LTOE_Y":85,"LTOE_Z":86,
              "LANK_X":87,"LANK_Y":88,"LANK_Z":89, #29*3
              "Rightant_X":90,"Rightant_Y":91,"Rightant_Z":92,
              "Rightpost_X":93,"Rightpost_Y":94,"Rightpost_Z":95,
              "Leftant_X":96,"Leftant_Y":97,"Leftant_Z":98,
              "Leftpost_X":99,"Leftpost_Y":100,"Leftpost_Z":101}

In [8]:
#Sampling frequency: piano->250, violin->120
sf = 120 #violin
# sf = 250 #piano

In [9]:
def predict(model, input, device):
    model.eval()
    with torch.no_grad():
        input = torch.as_tensor(input).to(torch.float32).to(device)
        # print(target.shape)
        # target = torch.as_tensor(target).to(torch.float32).to(device)
        outputs = model(input)
        return outputs.cpu().numpy()

In [10]:
def column(matrix, i):
    return [row[i] for row in matrix]

def test_render_animation(fps, output, azim, prediction, ground_truth=None):
    prediction_array = np.asarray(prediction)
    print(prediction_array.size)
    limit = len(prediction_array)
    print("limit", limit)
    size = 6#6
    fps = 40

    # Skeleton layout
    parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                [8, 18], [8, 20], [13, 21], [13, 19],
                [5, 20], [5, 21], [7, 18], [7, 19],
                [18, 19], [19, 21], [21, 20], [20, 18], #waist
                [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
    # joints_right = [1, 2, 12, 13, 14]

    prediction_array[:, :, 2] += 0.1 #[:, :, 2]
    if ground_truth is not None:
        ground_truth[:, :, 2] += 0.1
        poses = {'Prediction': prediction_array,
                 'Ground_truth': ground_truth}
    else:
        poses = {'Prediction': prediction_array}
    

    fig = plt.figure()#(figsize=(size*len(poses), size))
    # ax_3d = []
    # lines_3d = []
    radius = 1#14 #3.7#
    # print(poses)
    for index, (title, data) in enumerate(poses.items()):
        ax = fig.add_subplot(1, len(poses), index + 1, projection='3d')
        ax.clear()
        print(data)
        ims = [] #每一 frame 都存
        for frame_index, each_frame in enumerate(data):
            # print("each_frame")
            # print(each_frame)
            ax.view_init(elev=15., azim=azim)
            ax.set_xlim3d([-radius/2, radius/2])
            ax.set_zlim3d([0, radius])
            ax.set_ylim3d([-radius/2, radius/2])
            ax.set_aspect('auto') #ax.set_aspect('equal')

            # print(title)
            points = ax.scatter(column(each_frame[:30], 0), column(each_frame[:30], 1), column(each_frame[:30], 2), cmap='jet', marker='o', label='body joint', color = 'black')
            points_2 = ax.scatter(column(each_frame[30:32], 0), column(each_frame[30:32], 1), column(each_frame[30:32], 2), cmap='jet', marker='o', label='body joint', color = 'blue')
            points_3 = ax.scatter(column(each_frame[32:34], 0), column(each_frame[32:34], 1), column(each_frame[32:34], 2), cmap='jet', marker='o', label='body joint', color = 'red')
            
            # ax.scatter(column(each_frame, 0), column(each_frame, 1), column(each_frame, 2), cmap='jet', marker='o', label='body joint')
            # ax.legend()
            # print("+++")
            
            parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                        [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                        [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                        [8, 18], [8, 20], [13, 21], [13, 19],
                        [5, 20], [5, 21], [7, 18], [7, 19],
                        [18, 19], [19, 21], [21, 20], [20, 18], #waist
                        [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                        [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                        [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                        [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                        [30, 31], [32, 33],  #instrument
                        # [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
            lines = []
            # draw line
            
            # lines = [ax.plot([each_frame[vs][0], each_frame[ve][0]],
            #                  [each_frame[vs][1], each_frame[ve][1]],
            #                  [each_frame[vs][2], each_frame[ve][2]]) for (vs, ve) in parents]
            line_num = len(parents)
            for idx, each_line in enumerate(parents):
                vec_start = each_frame[each_line[0]]
                vec_end = each_frame[each_line[1]]
                # print(vec_start)
                # print(vec_end)
                line_color = "black"
                if idx == line_num-2:
                    line_color = "blue"
                if idx == line_num-1:
                    line_color = "red"
                # ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]])
                
                temp, = ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]], color=line_color)
                lines.append(temp)

            # ax.figure.savefig('./test_pic/pic' + str(frame_index) + '.png', dpi=100, bbox_inches = 'tight')

            # ims.append([points])
            # image_frame = [points].extend(lines)
            ims.append([points]+[points_2]+[points_3]+lines) #TODO: try extend

            # plt.cla()
            # print("+++")

    anim = matplotlib.animation.ArtistAnimation(fig, ims, interval=1000/fps)

    if output.endswith('.mp4'):
        FFwriter = matplotlib.animation.FFMpegWriter(fps=fps, extra_args=['-vcodec', 'libx264'])
        anim.save(output, writer=FFwriter)
    elif output.endswith('.gif'):
        anim.save(output, fps=fps, dpi=100, writer='imagemagick')
    else:
        raise ValueError('Unsupported output format (only .mp4 and .gif are supported)')

In [11]:
def plot(audio_path, plot_path, prediction, sample_time, fps, name=""): #audio_path, plot_path, 
    # render_animation(fps, output='new_temp.mp4', azim=75, prediction=prediction)
    test_render_animation(fps, output='new_temp_' + name + '.mp4', azim=75, prediction=prediction)

    # # #merge with wav
    input_video = ffmpeg.input('new_temp_' + name + '.mp4')
    fluid_syn = FluidSynth()
    fluid_syn.midi_to_audio(audio_path, './output' + name + '.wav')
    input_audio = ffmpeg.input('./output' + name + '.wav')
    # output = ffmpeg.output(video, audio, plot_path, vcodec='copy', acodec='aac', strict='experimental')
    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(plot_path).run()
    # os.remove('new_temp_' + name + '.mp4')

In [12]:
def plot_wav(audio_path, plot_path, prediction, sample_time, fps, name): #audio_path, plot_path, 
    # render_animation(fps, output='new_temp.mp4', azim=75, prediction=prediction)
    test_render_animation(fps, output='new_temp_' + name + '.mp4', azim=75, prediction=prediction)

    # # #merge with wav
    input_video = ffmpeg.input('new_temp_' + name + '.mp4')
    input_audio = ffmpeg.input(audio_path)
    
    if "Flower" in name:
        isExisting = os.path.exists('./out_tempclip_' + name + '.wav')
        if isExisting != True:
            audio_cut = input_audio.audio.filter('atrim', start=64, end=84)
            audio_output = ffmpeg.output(audio_cut, 'out_tempclip_' + name + '.wav')
            ffmpeg.run(audio_output)
        input_audio = ffmpeg.input('./out_tempclip_' + name + '.wav')
    
    if "Elgar" in name:
        isExisting = os.path.exists('./out_tempclip_' + name + '.wav')
        if isExisting != True:
            audio_cut = input_audio.audio.filter('atrim', start=103, end=123) #, start=30, end=60
            audio_output = ffmpeg.output(audio_cut, './out_tempclip_' + name + '.wav')
            ffmpeg.run(audio_output)
        input_audio = ffmpeg.input('./out_tempclip_' + name + '.wav')
    # output = ffmpeg.output(video, audio, plot_path, vcodec='copy', acodec='aac', strict='experimental')
    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(plot_path).run()
    # os.remove('new_temp_' + name + '.mp4')

In [13]:
def reset_weights(model): # reset the weight every fold
    if isinstance(model, nn.LSTM) or isinstance(model, nn.Linear):
        model.reset_parameters()

In [14]:
class LSTM1(nn.Module):
    def __init__(self, output_dim, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.output_dim = output_dim #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, output_dim) #fully connected to determine output dim

        self.relu = nn.ReLU()

    def forward(self,x):
        # h0, c0 no time information
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state
        # Propagate input through LSTM
        # x is MIDI => [44, 512, 128]

        # hn is final state, run over the sequence length
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        # hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        # print("output.shape", output.shape)
        # print("hn.shape", hn.shape)
        # out = self.relu(hn)
        out = self.fc_1(output) #final
        return out
 

In [15]:
# Define the model architecture
input_size_midi = 128
input_size_audio = 156#156+128 #number of features
input_size_both = 284

hidden_size = 1024 #1024 #512 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
seq_len = 512

output_dim_with_anno = 115 #number of output classes
output_dim_no_anno = 102

batch_size_define_with_anno = 20#128
batch_size_define_no_anno = 128

learning_rate = 0.001#0.001

In [16]:
def model_predict(model_path, input_size, output_dim, test_data, filecode, loss_type):
    model = LSTM1(output_dim, input_size, hidden_size, num_layers, seq_len).to(device) #our lstm class
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(checkpoint['model_state_dict'])
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    model.eval()
    
    data_format = ""
    if input_size == 128:
        data_format = "MIDI"
    if input_size == 156:
        data_format = "Audio"
    if input_size == 284:
        data_format = "Both"
    
    NoY = ""
    if output_dim == 102:
        NoY = "no"
    if output_dim == 115:
        NoY = "yes"
    
    num_epochs = 0
    if loss_type == "mse":
        num_epochs = 500
    if loss_type == "new":
        num_epochs = 300
    
    full_prediction = pd.DataFrame()
    num_count = 0
    # read midi
    # test_dataloader = get_dataloader(test_datapath, batch_size=1)
    # for test_batch in test_data_list:
    with torch.no_grad():
        # first_target = torch.zeros(test_batch.shape[0],112)
        # print(first_target.shape)
        test_input = test_data[None, :]
        # test_target = first_target[None, :]
        print("test_input", test_input.shape)
        # print("test_target", test_target.shape)
        prediction = predict(model, test_input, device)
        
        # print(prediction.shape)
        
        prediction  = prediction[:, :, :102]
        print("prediction.shape", prediction.shape)
        
        # full_prediction.append(prediction)
        full_prediction = pd.DataFrame(prediction[0])
        print("full_prediction", full_prediction.shape)
        
        # prev_prediction = prediction[0][:-1][None, :]
        # print(prev_prediction.shape)
        
        Row_list_prediction =[]
    
        # Iterate over each row
        for index, rows in full_prediction.iterrows():
            #fill nan
            rows = rows.fillna(0)
            # Create list for the current row
            my_list = rows.values.tolist()
            # print(my_list)
            
            my_list_per3 = [my_list[i:i+3] for i in range(0, len(my_list), 3)]
            # append the list to the final list
            Row_list_prediction.append(my_list_per3)

        # print(len(Row_list_prediction), len(Row_list_prediction[0]),len(Row_list_prediction[0][0]))
        # UoE_violin_midi_path = "./UoE_violin_midi/violin_midi/vio01/"
        # if data_format == "MIDI":
        #     # MIDI input
        #     plot(UoE_violin_midi_path + "vio01_" + filecode + ".mid", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[:800], None, 40, filecode) #ow_list[0:900]
        # else:
        # wav input
        
        if "Elgar" in filecode:#4120
            plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[4120:4920], None, 40, filecode) #ow_list[0:900]

        if "Flower" in filecode:#2560
            plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[2560:3360], None, 40, filecode) #ow_list[0:900]
        
        # print("prediction.shape", prediction.shape)
        prediction_arr = np.array(Row_list_prediction)
        if not os.path.exists("./save_prediction/[" + data_format + "_" + NoY + "_anno](" + loss_type + ")"+str(num_layers)+"LSTM_hidden"+str(hidden_size)+'_'+str(num_epochs)+"epoch/"):
            os.makedirs("./save_prediction/[" + data_format + "_" + NoY + "_anno](" + loss_type + ")"+str(num_layers)+"LSTM_hidden"+str(hidden_size)+'_'+str(num_epochs)+"epoch/")
        both_data_output = open("./save_prediction/[" + data_format + "_" + NoY + "_anno](" + loss_type + ")"+str(num_layers)+"LSTM_hidden"+str(hidden_size)+'_'+str(num_epochs)+"epoch/prediction_"+
                                filecode +".pkl", 'wb')
        pickle.dump(prediction_arr, both_data_output)
        both_data_output.close()
        # formated_motion = prediction_format(full_prediction)
        # # # plot(formated_motion)
        # audio_path = test_music_list[num_count][0]
        # output_path = "test_output_" + filecode + ".mp4"
        # plot(formated_motion, audio_path, output_path, None, 10, filecode)
        num_count += 1

    # model.train()

In [17]:
no_anno_model_path = "./no_anno_model_save/"
with_anno_model_path = "./with_anno_model_save/"

no_anno_model_path_list = glob.glob(no_anno_model_path + "*.tar")


for eval_model_path in no_anno_model_path_list:
    for filecode in test_pieces:
        #[audio_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_23-05-02
        # eval_path.split(' ', )
        print(eval_model_path)
        input_size = 0
        if ('midi' in eval_model_path):
            input_size = 128
            test_data = piano_roll_dictionary[this_performer][filecode]
        if ('audio' in eval_model_path):
            input_size = 156
            test_data = audio_dictionary[this_performer][filecode]
        if ('both' in eval_model_path):
            input_size = 284
            test_data = np.concatenate(
            (piano_roll_dictionary[this_performer][filecode], audio_dictionary[this_performer][filecode]), axis=1)
            # test_data = motion_dictionary[this_performer][filecode]
        
        output_dim = 0
        if ('no_anno' in eval_model_path):
            output_dim = 102
        if ('with_anno' in eval_model_path):
            output_dim = 115
        
        loss_type = ""
        if ("total500" in eval_model_path):
            loss_type = "mse"
        if ("total300" in eval_model_path):
            loss_type = "new"
        print(loss_type)

        model_predict(eval_model_path, input_size, output_dim, test_data, filecode, loss_type)

./no_anno_model_save/[midi_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_06-18-09.tar
new


OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [ 2.2387e-02, -1.7220e-02,  2.3143e-02,  ...,  1.8405e-02,
          2.1236e-02, -1.6784e-02],
        [-1.5422e-02, -5.6844e-03,  1.3451e-03,  ..., -1.9124e-02,
          1.4527e-02, -3.9209e-03],
        [-5.6012e-04, -1.8845e-02,  2.0255e-02,  ...,  6.3918e-04,
          2.1597e-02,  5.5955e-03]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.3526, -0.4451,  0.2492,  ...,  0.2224,  0.2192, -0.4368],
        [ 0.0503,  0.2439,  0.2582,  ...,  0.0911, -0.1098,  0.4041],
        [ 0.1968,  0.4012,  0.1053,  ..., -0.3714, -0.3238,  0.0288],
        ...,
        [ 0.1233,  0.4862,  0.0470,  ..., -0.4186, -0.3679,  0.3941],

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

./no_anno_model_save/[midi_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_06-18-09.tar
new
OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [ 2.2387e-02, -1.7220e-02,  2.3143e-02,  ...,  1.8405e-02,
          2.1236e-02, -1.6784e-02],
        [-1.5422e-02, -5.6844e-03,  1.3451e-03,  ..., -1.9124e-02,
          1.4527e-02, -3.9209e-03],
        [-5.6012e-04, -1.8845e-02,  2.0255e-02,  ...,  6.3918e-04,
          2.1597e-02,  5.5955e-03]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.3526, -0.4451,  0.2492,  ...,  0.2224,  0.2192, -0.4368],
        [ 0.0503,  0.2439,  0.2582,  ...,  0.0911, -0.1098,  0.4041],
        [ 0.1968,  0.4012,  0.1053,  ..., -0.3714, -

In [ ]:
with_anno_model_path_list = glob.glob(with_anno_model_path + "*.tar")

for eval_model_path in with_anno_model_path_list:
    for filecode in test_pieces:
        #[audio_no_anno][total300_hs1024]LSTM_save_epoch_299_2023-09-01_23-05-02
        # eval_path.split(' ', )
        input_size = 0
        if ('midi' in eval_model_path):
            input_size = 128
            test_data = piano_roll_dictionary[this_performer][filecode]
        if ('audio' in eval_model_path):
            input_size = 156
            test_data = audio_dictionary[this_performer][filecode]
        if ('both' in eval_model_path):
            input_size = 284
            test_data = np.concatenate(
            (piano_roll_dictionary[this_performer][filecode], audio_dictionary[this_performer][filecode]), axis=1)
            # test_data = motion_dictionary[this_performer][filecode]
        
        output_dim = 0
        if ('no_anno' in eval_model_path):
            output_dim = 102
        if ('with_anno' in eval_model_path):
            output_dim = 115
        
        loss_type = ""
        if ('total500' in eval_model_path):
            loss_type = "mse"
        if ('total300' in eval_model_path):
            loss_type = "new"
            
        model_predict(eval_model_path, input_size, output_dim, test_data, filecode, loss_type)

OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [-2.2005e-03, -1.4138e-02,  2.1567e-02,  ...,  2.6931e-02,
         -1.0656e-02, -1.4090e-04],
        [ 2.9328e-02, -1.1218e-02, -1.4724e-02,  ...,  2.3250e-02,
         -1.9489e-02,  3.0060e-02],
        [ 1.2342e-02, -2.9460e-02, -1.1747e-02,  ..., -1.9196e-03,
         -6.8986e-03,  1.2311e-02]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.1561, -0.0902,  0.1600,  ..., -0.6004,  0.2462,  0.2367],
        [-0.1749, -0.0397, -0.1754,  ..., -0.5262, -0.1405,  0.0111],
        [ 0.0782, -0.5275, -0.3311,  ..., -0.2035,  0.1834, -0.1874],
        ...,
        [ 0.3344,  0.0308, -0.1157,  ...,  0.4482,  0.1918,  0.1961],

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

OrderedDict([('lstm.weight_ih_l0', tensor([[ 7.0600e-03, -3.0621e-02, -6.3491e-03,  ..., -1.2402e-02,
          2.6364e-02, -1.1718e-05],
        [ 2.3990e-02, -8.1953e-04,  2.0643e-02,  ...,  8.4266e-03,
          7.3693e-04, -2.0229e-02],
        [-3.0530e-02, -1.5380e-02,  2.8527e-02,  ...,  1.6522e-03,
          1.8220e-02,  1.1862e-02],
        ...,
        [-2.2005e-03, -1.4138e-02,  2.1567e-02,  ...,  2.6931e-02,
         -1.0656e-02, -1.4090e-04],
        [ 2.9328e-02, -1.1218e-02, -1.4724e-02,  ...,  2.3250e-02,
         -1.9489e-02,  3.0060e-02],
        [ 1.2342e-02, -2.9460e-02, -1.1747e-02,  ..., -1.9196e-03,
         -6.8986e-03,  1.2311e-02]], device='cuda:0')), ('lstm.weight_hh_l0', tensor([[ 0.1561, -0.0902,  0.1600,  ..., -0.6004,  0.2462,  0.2367],
        [-0.1749, -0.0397, -0.1754,  ..., -0.5262, -0.1405,  0.0111],
        [ 0.0782, -0.5275, -0.3311,  ..., -0.2035,  0.1834, -0.1874],
        ...,
        [ 0.3344,  0.0308, -0.1157,  ...,  0.4482,  0.1918,  0.1961],

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

In [ ]:
for filecode in test_pieces:
    ground_truth = pd.DataFrame(motion_dictionary[this_performer][filecode][:, :102])
    print(ground_truth.shape)
    
    Row_list_prediction =[]
    
    # Iterate over each row
    for index, rows in ground_truth.iterrows():
        #fill nan
        rows = rows.fillna(0)
        # Create list for the current row
        my_list = rows.values.tolist()
        # print(my_list)
        
        my_list_per3 = [my_list[i:i+3] for i in range(0, len(my_list), 3)]
        # append the list to the final list
        Row_list_prediction.append(my_list_per3)

    print(len(Row_list_prediction))
    
    if "Elgar" in filecode:#4120
        plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[ground_truth_40fps]video_" + filecode + ".mp4", Row_list_prediction[4120:4920], None, 40, filecode)
        # plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[4120:4920], None, 40, filecode) #ow_list[0:900]

    if "Flower" in filecode:#2560
        plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[ground_truth_40fps]video_" + filecode + ".mp4", Row_list_prediction[2560:3360], None, 40, filecode)
        # plot_wav(UoE_violin_audio_path + "vio01_" + filecode + ".wav", "./[" + data_format + "_" + NoY + "_anno](" + loss_type + ")video_" + filecode + "_predict_hs"+str(hidden_size)+'_'+str(num_epochs)+"epoch"+".mp4", Row_list_prediction[2560:3360], None, 40, filecode) #ow_list[0:900]


(7355, 102)
7355
81600
limit 800
[[[ 9.86281581e-02  1.00470039e-01  1.10000000e+00]
  [ 2.96077901e-02  7.49391786e-02  1.10371394e+00]
  [ 1.32824395e-01  2.55640088e-02  1.10631972e+00]
  ...
  [ 4.50791465e-02  8.51754456e-02  9.78756378e-01]
  [ 1.96899723e-01  4.01596611e-01  4.98375456e-01]
  [ 2.23522187e-01  9.47548453e-03  5.25989232e-01]]

 [[ 1.02621756e-01  1.01007444e-01  1.10000000e+00]
  [ 3.36823543e-02  7.54023837e-02  1.10808862e+00]
  [ 1.37051871e-01  2.58389306e-02  1.10369123e+00]
  ...
  [ 4.38427516e-02  8.61169294e-02  9.81801927e-01]
  [ 2.46030692e-01  3.91626588e-01  3.59772227e-01]
  [ 2.13969448e-01 -5.51447519e-03  5.47534202e-01]]

 [[ 1.02246885e-01  1.01015463e-01  1.10000000e+00]
  [ 3.33377646e-02  7.49792436e-02  1.10680424e+00]
  [ 1.36790920e-01  2.58455707e-02  1.10440325e+00]
  ...
  [ 4.52149511e-02  8.58205556e-02  9.80856451e-01]
  [ 2.33580462e-01  3.94652409e-01  3.97129584e-01]
  [ 2.16638927e-01 -1.55837459e-03  5.40953702e-01]]

 ...

 

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

(4618, 102)
4618
81600
limit 800
[[[ 0.1141281   0.10652966  1.1       ]
  [ 0.04563127  0.08174538  1.10540524]
  [ 0.14732255  0.03212557  1.10010427]
  ...
  [ 0.06134782  0.09651684  0.96821612]
  [ 0.31142773  0.39869451  0.58343764]
  [ 0.22211321  0.00909644  0.51137118]]

 [[ 0.12898226  0.0995697   1.1       ]
  [ 0.05765211  0.0824332   1.10218621]
  [ 0.1529517   0.020031    1.09768721]
  ...
  [ 0.07945836  0.10205438  0.96581072]
  [ 0.43814217  0.35751215  0.56315185]
  [ 0.21876066 -0.0130861   0.50862323]]

 [[ 0.12570022  0.10170834  1.1       ]
  [ 0.05491018  0.08326911  1.103131  ]
  [ 0.1513396   0.02321888  1.09877958]
  ...
  [ 0.07454732  0.10077074  0.96672434]
  [ 0.40025401  0.37318804  0.56915765]
  [ 0.22087523 -0.0052466   0.50962083]]

 ...

 [[ 0.16276921  0.12214388  1.1       ]
  [ 0.09925985  0.08583837  1.11760585]
  [ 0.21124088  0.05367069  1.10895512]
  ...
  [ 0.11151492  0.0859464   0.97512394]
  [-0.05878035  0.02725975  1.0450201 ]
  [ 0.20943

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55